In [24]:
from collections import defaultdict
from collections import deque
import sys

Class Node -> to store all values that corresponds to the node/id.
All necessary variables are declared below:

In [25]:
class Node:
    def __init__(self,id,name,dur,es,ef,ls,lf) -> None:
        self.name = name
        self.id = id ; self.du = dur
        self.es = es; self.ef = ef
        self.ls = ls; self.lf = lf
    
nodes = {}
visited = {}
q = deque()
graph = defaultdict(list)
criticalPath = []
totalDuration = 0
leaves = {} 

Reading input file and Storing all the corresponding value.
nodes dictionary used to store the object of Node class.
All the id appended to the queue which have no predecessors.

In [26]:
path = './input.txt'
for lines in open(path):
    line = lines.rstrip('\n').split(',')
    id = int(line[0]); name = line[1]; duration = int(line[2])
    ef = 0
    if len(line) == 4:
        predecessors = line[3].split(';')
        for x in predecessors:
            x = int(x)
            graph[x].append(id)
            graph[id].append(x)
    else:
        q.append(id)
        ef = duration

    nodes[id] = Node(id,name,duration,0,ef,0,sys.maxsize)
    visited[id] = 0
    leaves[id] = 0

ForwardPass:
    Traverse through the graph using BFS.
    Update es and ef --- es is updated with maximum value of its predecessors.
    outDegree is used to find the leaves from which we'll start backwardPass.

In [27]:
while q:
    outDegree = 0
    u = q.popleft()
    ef = nodes[u].ef
    for v in graph[u]:
        if visited[v] != 1:
            nodes[v].es = max(ef,nodes[v].es)
            nodes[v].ef = nodes[v].es + nodes[v].du
            q.append(v)
            outDegree += 1

    if outDegree == 0 and visited[u] == 0:
        leaves[u] = 1
        totalDuration = max(totalDuration,nodes[u].ef)
        
    visited[u] = 1

BackwardPass:
    All leaf nodes are added to the queue and corresponding ls , lf is calculated.
    Using BFS Graph is traversed in backward direction.
    lf is calculated as minimum(ls) of its successors.

In [28]:
for node,leaf in leaves.items():
    if leaf:
        nodes[node].lf = totalDuration
        nodes[node].ls = nodes[node].lf - nodes[node].du
        q.append(node)
    visited[node] = 0

while q:
    u = q.popleft()
    visited[u] = 1
    ls = nodes[u].ls
    for v in graph[u]:
        if visited[v] != 1:
            nodes[v].lf = min(nodes[v].lf,ls)
            nodes[v].ls = nodes[v].lf - nodes[v].du
            q.append(v)

In [29]:
def printNodes(nodes):
    for key,value in nodes.items():
        print('id = {},node = {},duration = {},ES = {},EF = {},LS = {},LF = {}'
            .format(key,value.name,value.du,value.es,value.ef,value.ls,value.lf))

In [30]:
printNodes(nodes)

id = 10,node = A,duration = 7,ES = 0,EF = 7,LS = 0,LF = 7
id = 20,node = B,duration = 10,ES = 7,EF = 17,LS = 7,LF = 17
id = 25,node = C,duration = 3,ES = 7,EF = 10,LS = 14,LF = 17
id = 30,node = D,duration = 5,ES = 7,EF = 12,LS = 12,LF = 17
id = 40,node = E,duration = 5,ES = 17,EF = 22,LS = 17,LF = 22


In [31]:
for key,value in nodes.items():
    if value.es == value.ls:
        criticalPath.append(value.name)

In [32]:
n = len(criticalPath)
for i in range(n):
    print(criticalPath[i],end = '')
    if i != n - 1:
        print('->',end='')

A->B->E